# jQAssistant Demo
* Clone https://github.com/JavaOnAutobahn/spring-petclinic
* Build software
 * `mvn install`
* Start Neo4j server
 * `mvn jqassistant:server`
* Open browser
 * http://localhost:7474/browser/
* jQAssistant documentation: https://jqassistant.github.io/jqassistant/doc/1.10.0/manual/index.html
 

## Example Cypher Queries

### Setup connection to a running N4o4j database

In [2]:
from neo4j import GraphDatabase

URI = "bolt://localhost"
AUTH = ("neo4j", "neo4j")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()
session = driver.session()

### List TOP 10 class with the most methods

In [3]:
query="""
MATCH 
  (t:Type)-[:DECLARES]->(m:Method)
RETURN t.fqn as Typ, COUNT(m) as methodCount
ORDER BY methodCount DESC
LIMIT 10
"""
session.run(query).to_df()

,Typ,methodCount
0,org.springframework.samples.petclinic.model.Owner,21
1,org.springframework.samples.petclinic.model.Pet,16
2,org.springframework.samples.petclinic.web.Owne...,13
3,org.springframework.samples.petclinic.service....,12
4,org.springframework.samples.petclinic.service....,11
5,org.springframework.samples.petclinic.model.Vet,11
6,org.springframework.samples.petclinic.reposito...,10
7,org.springframework.samples.petclinic.model.Visit,10
8,org.springframework.samples.petclinic.web.PetC...,9
9,org.springframework.samples.petclinic.web.Owne...,9


### Global  written variables

In [4]:
query="""
MATCH (c:Class)-[:DECLARES]->(f:Field)<-[w:WRITES]-(m:Method)
WHERE 
    EXISTS(f.static) AND NOT EXISTS(f.final)
RETURN 
    c.name as InClass, 
    m.name as theMethod, 
    w.lineNumber as writesInLine, 
    f.name as toStaticField
"""
session.run(query).to_df()

,InClass,theMethod,writesInLine,toStaticField
0,OwnerController,processFindForm,112,ownersIndexes
1,OwnerController,processFindForm,112,ownersIndexes
2,BrokenSingleton,getInstance,11,INSTANCE


### Recrusive data strutures

In [5]:
query="""
MATCH (c:Class)-[:DECLARES]->(m:Method)-[:INVOKES]->(m)
RETURN c.name, m.name
"""
session.run(query).to_df()

,c.name,m.name
0,RecursiveClass,recursive
1,RecursiveDbCallsExample,loadParent


### Aggregation of values across business subdomains

In [6]:
query="""
MATCH 
  (t:Type)-[:BELONGS_TO]->(s:Business:Subdomain),
  (t)-[:HAS_CHANGE]->(ch:Change)
RETURN 
  s.name as ASubdomain,
  COUNT(DISTINCT t) as Types,
  COUNT(DISTINCT ch) as Changes
ORDER BY Types DESC
"""
session.run(query).to_df()

,ASubdomain,Types,Changes
0,Pet,15,209
1,Visit,9,119
2,Vet,8,117
3,Owner,7,130
4,Clinic,6,102
5,Person,1,5
6,Specialty,1,7


### Code dependencies between business subdomains

In [7]:
query="""
MATCH
  (t1:Type)-[:BELONGS_TO]->(s1:Business:Subdomain),
  (t2:Type)-[:BELONGS_TO]->(s2:Business:Subdomain),
  (t1)-[d:DEPENDS_ON]->(t2)
WHERE
  s1 <> s2
WITH
  s1, s2, count(d) as weight
RETURN
  s1.name as Subdomain, collect(s2.name) as Dependencies
ORDER BY
  Subdomain
"""
session.run(query).to_df()

,Subdomain,Dependencies
0,Clinic,"[Owner, Pet, Specialty, Vet, Visit]"
1,Owner,"[Clinic, Person, Pet, Visit]"
2,Pet,"[Clinic, Owner, Visit]"
3,Vet,"[Clinic, Person, Specialty]"
4,Visit,"[Clinic, Pet]"


### Cyclic dependencies between packages

In [8]:
query="""
MATCH
    (p1:Package)-[:DEPENDS_ON]->(p2:Package),
    path=shortestPath((p2)-[:DEPENDS_ON*]->(p1))
WHERE
    p1<>p2
RETURN
    p1 AS Package, EXTRACT(p IN nodes(path) | p.fqn) AS Cycle
ORDER BY
    Package.fqn
"""
session.run(query).to_df()

,Package,Cycle
0,"(name, fqn, fileName)",[org.springframework.samples.petclinic.reposit...
1,"(name, fileName, fqn)","[org.springframework.samples.petclinic.util, o..."
2,"(name, fileName, fqn)","[org.springframework.samples.petclinic.model, ..."
3,"(name, fileName, fqn)","[org.springframework.samples.petclinic.web, or..."
4,"(name, fqn, fileName)","[org.springframework.samples.petclinic.model, ..."
5,"(name, fqn, fileName)",[org.springframework.samples.petclinic.service...
